In [22]:
import pandas as pd
import requests
import json
import csv
import numpy as np
from datetime import datetime

In [37]:
df_secciones = pd.read_csv("data/Elypsis/Secciones electorales.csv")

In [38]:
path = "data/Elypsis/"
df_intencion = pd.read_csv(path + 'Post PASO - 8 raw.csv', header=[0])
df_temp = pd.DataFrame()

In [39]:
df_intencion.head(1)

,respondent_id,collector_id,date_created,date_modified,ip_address,email_address,first_name,last_name,custom_1,condiciones,...,¿Qué es lo que más le preocupa? Inseguridad/delincuencia,¿Qué es lo que más le preocupa? Perder el trabajo,¿Qué es lo que más le preocupa? Corrupción,¿Qué es lo que más le preocupa? Dificultades para pagar créditos/tarjeta,¿Qué es lo que más le preocupa? Se siguen dando subsidios a quienes no lo merece,¿Qué es lo que más le preocupa? Incertidumbre sobre la situación económica del país,¿Qué es lo que más le preocupa? No se detienen los piquetes/cortes de calle/rutas,¿Qué es lo que más le preocupa? Aumento en tarifas de servicios públicos,¿Qué es lo que más le preocupa? Mala calidad de la educación,¿Qué es lo que más le preocupa? Drogas en el barrio
0,10947625474,243568425,8/25/2019 7:59,8/25/2019 8:05,186.13.115.88,NaN,NaN,NaN,NaN,Acepto las condiciones,...,Me preocupa mucho,Me preocupa mucho,Me preocupa mucho,Me preocupa mucho,Me preocupa,Me preocupa mucho,Me preocupa mucho,Me preocupa mucho,Me preocupa mucho,Me preocupa mucho


In [40]:
df_intencion.columns

Index(['respondent_id', 'collector_id', 'date_created', 'date_modified',
       'ip_address', 'email_address', 'first_name', 'last_name', 'custom_1',
       'condiciones', 'Nombre (Opcional) ', 'Email ', 'Usted es ', 'Edad ',
       '¿Cuál es su nivel de estudio? ',
       'Indique qué opción se aproxima más a su situación actual: ',
       'Indique en que sector trabaja (ejemplo: Construcción / Industria / Comercio)?',
       'intencion_presidencial',
       '¿Qué tan dispuesto estaría a votar a cada una de las formulas? Alberto Fernandez - Cristina Kirchner',
       '¿Qué tan dispuesto estaría a votar a cada una de las formulas? Mauricio Macri - Miguel Pichetto',
       '¿Qué tan dispuesto estaría a votar a cada una de las formulas? Roberto Lavagna - Juan Manuel Urtubey',
       '¿Qué tan dispuesto estaría a votar a cada una de las formulas? Nicolás del Caño - Romina del Plá',
       '¿Qué tan dispuesto estaría a votar a cada una de las formulas? José Luis Espert - Luis Rosales',
   

In [41]:
provincias = {'Interior Buenos Aires' : 'Int. PBA',  'Misiones': 'Resto',  'Córdoba': 'Resto',  'San Luis': 'Resto', 'Gran Buenos Aires': 'GBA',
 'Entre Ríos': 'Resto',  'Salta': 'Resto',  'Río Negro': 'Resto', 'Santiago del estero': 'Resto',  'Chaco': 'Resto',  'Mendoza': 'Resto', 
 'CABA': 'CABA',  'Tucuman': 'Resto', 'Santa Fé': 'Resto',  'La Pampa': 'Resto',  'Chubut': 'Resto',  'Catamarca': 'Resto', 
 'Corrientes': 'Resto', 'La Rioja': 'Resto',  'Jujuy': 'Resto',  'Formosa': 'Resto',  'San Juan': 'Resto',  'Neuquén': 'Resto',
 'Santa Cruz': 'Resto',  'Tierras del fuego': 'Resto'}

In [42]:
df_intencion = df_intencion.rename(columns={'Para votante de PBA - Si las elecciones a gobernador fueran hoy ¿A qué fórmula votaría? Response':'intencion_PBA'})

In [43]:
df_intencion = df_intencion.merge(df_secciones, how='left', left_on='partidoPBA', right_on='Localidad')

In [44]:
# remove ineffective
df_intencion = df_intencion[df_intencion['Edad '].notnull()]
df_intencion = df_intencion[df_intencion['Email '].notnull()]
df_intencion = df_intencion[df_intencion['Edad '] != 'Menos de 16']

df_intencion = df_intencion.drop_duplicates(subset=['Email '], keep='first')
df_intencion['¿Cuál es su nivel de estudio? '].loc[df_intencion['¿Cuál es su nivel de estudio? '] == 'Universitario completo'] = 'Universitario incompleto o terciario'
df_intencion['¿Cuál es su nivel de estudio? '].loc[df_intencion['¿Cuál es su nivel de estudio? '] == 'Universitario incompleto o terciario'] = 'Univ. completo o incompleto'
df_intencion['¿En qué provincia vota? '] = df_intencion['¿En qué provincia vota? '].replace(provincias)

df_intencion['intencion_presidencial'].loc[df_intencion['intencion_presidencial'] == 'https://www.infobae.com/new-resizer/5VwRyN-qKDVOTkPIT3mjiJ5xK0g=/750x0/filters:quality(100)/s3.amazonaws.com/arc-wordpress-client-uploads/infobae-wp/wp-content/uploads/2019/07/02173410/boleta-alberto-fernandez-cristina-kirchner.jpeg'] = 'AF-CFK'
df_intencion['intencion_presidencial'].loc[df_intencion['intencion_presidencial'] == 'https://www.infobae.com/new-resizer/0-sMpWhIQbEWXAYdYK4tOppww0s=/750x0/filters:quality(100)/s3.amazonaws.com/arc-wordpress-client-uploads/infobae-wp/wp-content/uploads/2019/07/02122432/boleta-juntos-por-el-cambio-macri-pichetto-SF-3.jpeg'] = 'MM-MP'
df_intencion['intencion_PBA'].loc[df_intencion['intencion_PBA'] == 'https://gallery.mailchimp.com/6b1e9fcb4df3eaee1cd635f50/images/4d63a1a0-ca04-4574-a213-1751fc95c32c.png'] = 'AK'
df_intencion['intencion_PBA'].loc[df_intencion['intencion_PBA'] == 'https://gallery.mailchimp.com/6b1e9fcb4df3eaee1cd635f50/images/990d24cd-8629-4138-9ac7-b9448b5ea65b.png'] = 'Vidal'
df_intencion['polarizado'] = False

df_intencion['polarizado'].loc[df_intencion['intencion_presidencial'] == 'MM-MP'] = True
df_intencion['polarizado'].loc[df_intencion['intencion_presidencial'] == 'AF-CFK'] = True

df_intencion['date_created'] = pd.to_datetime(df_intencion['date_created'])
df_intencion['semana2'] = round(((df_intencion['date_created'] - datetime.strptime('2016-01-01', '%Y-%m-%d')) / np.timedelta64(1, 'W')), 0) + 1

In [68]:
def definir_ponderadores_rake(dt_imagen, variable):
    if variable != "":
        dt_imagen = dt_imagen[dt_imagen[variable].notnull()]
        
    if 'pondera_base' not in dt_imagen:
        dt_imagen['pondera_base'] = 1

    for x in range(0, 10):
        for p_id, p_info in rake.items():
            print("campo:", p_id)
            dt_imagen_agg = dt_imagen.groupby(['semana2', p_id]).agg({'pondera_base': 'sum'})
            # print(dt_imagen_agg)
            share_semana = dt_imagen_agg.groupby(level=0).apply(lambda x: x / float(x.sum()))
            # print(share_semana)
            share_semana = share_semana.reset_index().rename(columns = {'pondera_base': 'share_base'})

            dt_imagen = pd.merge(dt_imagen, share_semana,  how='left', left_on=['semana2',p_id], right_on = ['semana2',p_id])

            dt_imagen['target_share'] = 0
            for key in p_info:
                print(str(key) + ':', p_info[key])    
                dt_imagen['target_share'].loc[dt_imagen[p_id] == key] = p_info[key]

            dt_imagen['pondera_base'] =  dt_imagen['pondera_base'] / dt_imagen['share_base'] * dt_imagen['target_share']
            # print(dt_imagen)
            
            del dt_imagen['share_base']
            del dt_imagen['target_share']

    return dt_imagen

In [ ]:
# rake = {
#             'Usted es ': {'Hombre': 0.479, 'Mujer': 0.521},
#             'Edad ':{'Entre 16 y 30': 0.311, 'entre 31 y 50': 0.323, 'entre 51 y 65': 0.193, 'Mas de 65 años': 0.136},
#             '¿Cuál es su nivel de estudio? ':{'Secundario Incompleto': 0.537, 'Secundario completo':0.208, 'Univ. completo o incompleto':0.255},
#             'polarizado': {True: 0.72, False: 0.28}
# }

rake = {
            'Usted es ': {'Hombre': 0.479, 'Mujer': 0.521},
            'Edad ':{'Entre 16 y 30': 0.311, 'entre 31 y 50': 0.323, 'entre 51 y 65': 0.193, 'Mas de 65 años': 0.136},
            '¿Cuál es su nivel de estudio? ': {'Secundario Incompleto': 0.537, 'Secundario completo':0.208, 'Univ. completo o incompleto':0.255} # 文化水平
}

# 性别，年龄，教育程度迭代
# keep location firstly
df_intencion['semana2'] = df_intencion['¿En qué provincia vota? '] # Location
df_intencion_n = definir_ponderadores_rake(df_intencion, '')

# 位置迭代
df_intencion_n['semana2'] = 1
rake = { '¿En qué provincia vota? ': {'Int. PBA': 0.13, 'Resto': 0.55, 'GBA': 0.24, 'CABA': 0.08} }
df_intencion_n = definir_ponderadores_rake(df_intencion_n, '')
df_intencion_n.head()

In [55]:
df_intencion_n.loc[
            (df_intencion_n['¿Que imagen tiene de....? Axel kicillof']=='Buena') &
            (df_intencion_n['intencion_presidencial']=='AF-CFK'), 
            'intencion_PBA'
            ] = "AK"

df_intencion_n.loc[
            (df_intencion_n['¿Que imagen tiene de....? Axel kicillof']=='Muy buena') &
            (df_intencion_n['intencion_presidencial']=='AF-CFK'), 
            'intencion_PBA'
            ] = "AK"

df_intencion_n.loc[
            (df_intencion_n['¿Que imagen tiene de....? María Eugenia Vidal']=='Buena') &
            (df_intencion_n['intencion_presidencial']=='MM-MP'), 
            'intencion_PBA'
            ] = "Vidal"

df_intencion_n.loc[
            (df_intencion_n['¿Que imagen tiene de....? María Eugenia Vidal']=='Muy buena') &
            (df_intencion_n['intencion_presidencial']=='MM-MP'), 
            'intencion_PBA'
            ] = "Vidal"

In [56]:
imagen = {
    '¿Que imagen tiene de....? Mauricio Macri',
    '¿Que imagen tiene de....? Alberto Fernandez',
    '¿Que imagen tiene de....? Cristina Fernandez de Kirchner',
    '¿Que imagen tiene de....? María Eugenia Vidal',
    '¿Que imagen tiene de....? Axel kicillof',
    '¿Que imagen tiene de....? Miguel Ángel Pichetto',
    '¿Que imagen tiene de....? José Luis Espert',
    '¿Que imagen tiene de....? Roberto Lavagna',
    '¿Que imagen tiene de....? Juan Manuel Urtubey',
    '¿Que imagen tiene de....? Elisa Carrio',
    '¿Que imagen tiene de....? Martín Lousteau'
}

df_intencion_n['no sabe'] = 0
for x in imagen:
    df_intencion_n.loc[df_intencion_n[x] == 'No sabe', 'no sabe'] += 1 
    df_intencion_n.loc[df_intencion_n[x] == 'No sabe quien es', 'no sabe'] += 1 

df_intencion_n['sabealgo'] = (df_intencion_n['no sabe'] > 4) 
df_intencion_n['pondera_nosabe'] = df_intencion_n['pondera_base'] * (1+0.4*df_intencion_n['sabealgo'])

In [58]:
df_intencion_n['date_created'] = pd.to_datetime(df_intencion_n['date_created'])     

In [59]:
outputfile = 'tmp.csv'
df_intencion_n.reset_index().to_csv(outputfile)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline 
text = df_intencion[df_intencion['Defina en una palabra a Cristina Kirchner'].notnull()]['Defina en una palabra a Cristina Kirchner'].values
wordcloud = WordCloud(
    width = 300,
    height = 200,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()